# Fedbiomed Researcher base example

Use for developing (autoreloads changes made across packages)

In [1]:
%load_ext autoreload
%autoreload 2

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default) to add MNIST to the node
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.

## Define an experiment model and parameters"

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [6]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlan, self).__init__(model_args)
        
        
        self.model = self.make_model()
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms"]
        
        self.add_dependency(deps)

    def make_model(self):
        model = nn.Sequential(nn.Conv2d(1, 32, 3, 1),
                                  nn.ReLU(),
                                  nn.Conv2d(32, 64, 3, 1),
                                  nn.ReLU(),
                                  nn.MaxPool2d(2),
                                  nn.Dropout(0.25),
                                  nn.Flatten(),
                                  nn.Linear(9216, 128),
                                  nn.ReLU(),
                                  nn.Dropout(0.5),
                                  nn.Linear(128, 10),
                                  nn.LogSoftmax(dim=1))
        return model
        
        
    def forward(self, x):

        return self.model(x)

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [7]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100, # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
    'DP_args': {'type' : 'local', 'sigma': 0.1, 'clip': 1.},
}

## Declare and run the experiment

- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `round_limit` rounds, applying the `node_selection_strategy` between the rounds

In [8]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-04-01 10:52:06,486 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
04/01/2022 10:52:06:INFO:Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-04-01 10:52:06,494 fedbiomed INFO - log from: node_4084c19b-0f4d-474c-9d11-ddfd8f7f84dc / DEBUG - Message received: {'researcher_id': 'researcher_024bc6ca-055d-406b-bdf2-68d99ef76a7c', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
04/01/2022 10:52:06:INFO:log from: node_4084c19b-0f4d-474c-9d11-ddfd8f7f84dc / DEBUG - Message received: {'researcher_id': 'researcher_024bc6ca-055d-406b-bdf2-68d99ef76a7c', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-04-01 10:52:16,492 fedbiomed INFO - Node selected for training -> node_4084c19b-0f4d-474c-9d11-ddfd8f7f84dc
04/01/2022 10:52:16:INFO:Node selected for training -> node_4084c19b-0f4d-474c-9d11-ddfd8f7f84dc
2022-04-01 10:52:16,506 fedbiomed DEBUG - Model file has been saved: /Users/mlorenzi/works/temp/fedbiomed/var/ex

Let's start the experiment.

By default, this function doesn't stop until all the `round_limit` rounds are done for all the nodes

In [9]:
exp.run()

2022-04-01 10:52:26,289 fedbiomed INFO - Sampled nodes in round 0 ['node_4084c19b-0f4d-474c-9d11-ddfd8f7f84dc']
04/01/2022 10:52:26:INFO:Sampled nodes in round 0 ['node_4084c19b-0f4d-474c-9d11-ddfd8f7f84dc']
2022-04-01 10:52:26,290 fedbiomed INFO - Send message to node node_4084c19b-0f4d-474c-9d11-ddfd8f7f84dc - {'researcher_id': 'researcher_024bc6ca-055d-406b-bdf2-68d99ef76a7c', 'job_id': '0c52cbb9-2071-4c83-87ff-bc01c2e74001', 'training_args': {'test_ratio': 0.0, 'test_on_local_updates': False, 'test_on_global_updates': False, 'test_metric': None, 'test_metric_args': {}, 'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100, 'DP_args': {'type': 'local', 'sigma': 0.1, 'clip': 1.0}}, 'training': True, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/04/01/my_model_5c771152-4ce5-414b-b348-42855fa62795.py', 'params_url': 'http://localhost:8844/media/uploads/2022/04/01/aggregated_params_init_fae5d2a0-f86a-4eb1-b96a-dc

2022-04-01 10:52:26,492 fedbiomed INFO - log from: node_4084c19b-0f4d-474c-9d11-ddfd8f7f84dc / DEBUG - Using device cpu for training (cuda_available=False, gpu=False, gpu_only=False, use_gpu=False, gpu_num=None)
04/01/2022 10:52:26:INFO:log from: node_4084c19b-0f4d-474c-9d11-ddfd8f7f84dc / DEBUG - Using device cpu for training (cuda_available=False, gpu=False, gpu_only=False, use_gpu=False, gpu_num=None)
2022-04-01 10:52:26,855 fedbiomed INFO - log from: node_4084c19b-0f4d-474c-9d11-ddfd8f7f84dc / INFO - Train Epoch: 1 [0/60000 (0%)]	Loss: 2.266146
04/01/2022 10:52:26:INFO:log from: node_4084c19b-0f4d-474c-9d11-ddfd8f7f84dc / INFO - Train Epoch: 1 [0/60000 (0%)]	Loss: 2.266146
2022-04-01 10:52:26,856 fedbiomed INFO - Monitor: node_id=node_4084c19b-0f4d-474c-9d11-ddfd8f7f84dc epoch=1 iteration=0 Loss:2.266146421432495
04/01/2022 10:52:26:INFO:Monitor: node_id=node_4084c19b-0f4d-474c-9d11-ddfd8f7f84dc epoch=1 iteration=0 Loss:2.266146421432495
2022-04-01 10:52:30,174 fedbiomed INFO - log

2022-04-01 10:53:06,482 fedbiomed INFO - Nodes that successfully reply in round 0 ['node_4084c19b-0f4d-474c-9d11-ddfd8f7f84dc']
04/01/2022 10:53:06:INFO:Nodes that successfully reply in round 0 ['node_4084c19b-0f4d-474c-9d11-ddfd8f7f84dc']
2022-04-01 10:53:06,916 fedbiomed DEBUG - upload (HTTP POST request) of file /Users/mlorenzi/works/temp/fedbiomed/var/experiments/Experiment_0001/aggregated_params_13c2879b-877f-4612-b6a6-abbd44145545.pt successful, with status code 201
04/01/2022 10:53:06:DEBUG:upload (HTTP POST request) of file /Users/mlorenzi/works/temp/fedbiomed/var/experiments/Experiment_0001/aggregated_params_13c2879b-877f-4612-b6a6-abbd44145545.pt successful, with status code 201
2022-04-01 10:53:06,918 fedbiomed INFO - Saved aggregated params for round 0 in /Users/mlorenzi/works/temp/fedbiomed/var/experiments/Experiment_0001/aggregated_params_13c2879b-877f-4612-b6a6-abbd44145545.pt
04/01/2022 10:53:06:INFO:Saved aggregated params for round 0 in /Users/mlorenzi/works/temp/fedb

04/01/2022 10:53:07:INFO:log from: node_4084c19b-0f4d-474c-9d11-ddfd8f7f84dc / ERROR - Cannot initialize model parameters:Error(s) in loading state_dict for MyTrainingPlan:
	Missing key(s) in state_dict: "model.0.weight", "model.0.bias", "model.2.weight", "model.2.bias", "model.7.weight", "model.7.bias", "model.10.weight", "model.10.bias". 
	Unexpected key(s) in state_dict: "model._module.0.weight", "model._module.0.bias", "model._module.2.weight", "model._module.2.bias", "model._module.7.weight", "model._module.7.bias", "model._module.10.weight", "model._module.10.bias". 
2022-04-01 10:53:16,934 fedbiomed INFO - Downloading model params after training on node_4084c19b-0f4d-474c-9d11-ddfd8f7f84dc - from 
04/01/2022 10:53:16:INFO:Downloading model params after training on node_4084c19b-0f4d-474c-9d11-ddfd8f7f84dc - from 
2022-04-01 10:53:16,937 fedbiomed ERROR - FB604: repository error : bad URL when downloading file node_params_c265624a-714d-42ab-b58a-badf0c8a8e81.pt(details :Invalid U


--------------------
Fed-BioMed researcher stopped due to exception:
FB604: repository error : bad URL when downloading file node_params_c265624a-714d-42ab-b58a-badf0c8a8e81.pt(details :Invalid URL '': No scheme supplied. Perhaps you meant http://? )
--------------------


Local training results for each round and each node are available via `exp.training_replies()` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [ ]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1].data()
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies()[rounds - 1].dataframe()

Federated parameters for each round are available via `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", exp.aggregated_params().keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params()[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())


Feel free to run other sample notebooks or try your own models :D